In [2]:
import pandas as pd

Official Counts

In [19]:
optz_counts = pd.read_csv('Z:/Users/iperez/optum_qa/transformed_record_counts/agg_counts/optum_zip_table_counts.csv')[['year', 'quarter', 'table', 'row_count', 'pat_count']]
optz_counts = optz_counts.rename(columns={'row_count':'official_row_count', 'pat_count':'official_pat_count'})
optz_counts.head()

,year,quarter,table,official_row_count,official_pat_count
0,2018.0,q2,confinement,550495,407812.0
1,2018.0,q3,confinement,544459,403645.0
2,2018.0,q4,confinement,557870,412868.0
3,2019.0,q1,confinement,585867,428608.0
4,2019.0,q2,confinement,575437,421152.0


greenplum optum counts, done already on **db_counts\gp_optum_zip_counts.csv**

In [18]:
gp_optz_counts = pd.read_csv('db_counts\gp_optum_zip_counts.csv')[['year', 'quarter', 'table_source', 'row_count', 'pat_count']]
gp_optz_counts = gp_optz_counts.rename(columns={'table_source':'table', 'row_count':'gp_row_count', 'pat_count':'gp_pat_count'})
gp_optz_counts.head()

,year,quarter,table,gp_row_count,gp_pat_count
0,NaN,NaN,mbr_enroll,157260336,75815144
1,NaN,NaN,mbr_co_enroll,92445133,75815144
2,2007.0,q1,medical,82185012,7303888
3,2007.0,q2,medical,81383400,7126881
4,2007.0,q3,medical,81385579,7147116


sql server optum counts, recorded on **db_counts\local_optz_counts_021623.csv**

In [17]:
spc_optz_counts = pd.read_csv('db_counts\local_optz_counts_021623.csv')[['year', 'quarter', 'table_source', 'row_count', 'pat_count']]
spc_optz_counts = spc_optz_counts.rename(columns={'table_source':'table', 'row_count':'spc_row_count', 'pat_count':'spc_pat_count'})
spc_optz_counts.head()

,year,quarter,table,spc_row_count,spc_pat_count
0,NaN,NaN,mbr_enroll,155540055,75058818
1,NaN,NaN,mbr_co_enroll,91328393,75058818
2,2007.0,q1,medical,82185012,7303888
3,2007.0,q2,medical,81383400,7126881
4,2007.0,q3,medical,81385579,7147116


joining the three dataframes

In [23]:
optz_comparisons = optz_counts.merge(gp_optz_counts, how='inner', on=['year', 'quarter', 'table'])
optz_comparisons = optz_comparisons.merge(spc_optz_counts, how='inner', on=['year', 'quarter', 'table'])
optz_comparisons.head()

,year,quarter,table,official_row_count,official_pat_count,gp_row_count,gp_pat_count,spc_row_count,spc_pat_count
0,2018.0,q2,confinement,550495,407812.0,550495,407812,549977,407700
1,2018.0,q3,confinement,544459,403645.0,544459,403645,544462,403652
2,2018.0,q4,confinement,557870,412868.0,557870,412868,557882,412875
3,2019.0,q1,confinement,585867,428608.0,585867,428608,585890,428633
4,2019.0,q2,confinement,575437,421152.0,575437,421152,575430,421146


In [24]:
optz_comparisons['official_gp_row_diff'] = optz_comparisons['official_row_count'] - optz_comparisons['gp_row_count']
optz_comparisons['official_gp_pat_diff'] = optz_comparisons['official_pat_count'] - optz_comparisons['gp_pat_count']
optz_comparisons['official_spcrow_diff'] = optz_comparisons['official_row_count'] - optz_comparisons['spc_row_count']
optz_comparisons['official_spc_pat_diff'] = optz_comparisons['official_pat_count'] - optz_comparisons['spc_pat_count']
optz_comparisons.head()

,year,quarter,table,official_row_count,official_pat_count,gp_row_count,gp_pat_count,spc_row_count,spc_pat_count,official_gp_row_diff,official_gp_pat_diff,official_spcrow_diff,official_spc_pat_diff
0,2018.0,q2,confinement,550495,407812.0,550495,407812,549977,407700,0,0.0,518,112.0
1,2018.0,q3,confinement,544459,403645.0,544459,403645,544462,403652,0,0.0,-3,-7.0
2,2018.0,q4,confinement,557870,412868.0,557870,412868,557882,412875,0,0.0,-12,-7.0
3,2019.0,q1,confinement,585867,428608.0,585867,428608,585890,428633,0,0.0,-23,-25.0
4,2019.0,q2,confinement,575437,421152.0,575437,421152,575430,421146,0,0.0,7,6.0


In [27]:
optz_comparisons.sort_values(['year', 'quarter', 'table']).to_excel('optz_comparisons.xlsx', index=False)